In [17]:
from poker_dataset import PokerDataset, PokerRound
from poker_test import get_models_accuracy
from poker_baseline_model import baseline_model
from poker_model import CustomFCNetwork
from helper import ACTION_NAMES

import torch

In [18]:
import importlib

importlib.reload(importlib.import_module('poker_dataset'))
importlib.reload(importlib.import_module('poker_test'))
importlib.reload(importlib.import_module('poker_baseline_model'))
importlib.reload(importlib.import_module('poker_model'))
importlib.reload(importlib.import_module('helper'))

<module 'helper' from 'c:\\Users\\kctak\\Documents\\Code\\PokerAI\\APS360 Project\\Poker-AI\\Poker Decision Model\\helper.py'>

In [2]:
filepath = r'C:\Users\kctak\Documents\Code\PokerAI\APS360 Progress\poker_data'
poker_dataset = PokerDataset(filepath, "IlxxxlI", balance_data=True)
print({action_name: len(indices) for action_name, indices in poker_dataset.output_indices.items()})

Processing Items: 100%|██████████| 50664/50664 [00:35<00:00, 1441.13it/s]

{'calls': 39175, 'raises': 39175, 'bets': 0, 'folds': 39175, 'checks': 39175, 'allin': 39175}


In [3]:
import torch.utils

test_frac = 0.15
val_frac = 0.15
train_frac = 1 - test_frac - val_frac

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(poker_dataset, [train_frac, val_frac, test_frac])

In [4]:
# Load trained model 
hidden_shape = [256, 128]
trained_model_path = r'Results2\poker_model.pth'

model = CustomFCNetwork(hidden_shape=hidden_shape)
model.load_state_dict(torch.load(trained_model_path))

<All keys matched successfully>

In [5]:
model_tracker, baseline_tracker = get_models_accuracy(model, baseline_model, test_dataset)

In [6]:
# model_tracker = {action_name: {'correct': [], 'wrong': []} for action_name in ACTION_NAMES}
# baseline_tracker = {action_name: {'correct': [], 'wrong': []} for action_name in ACTION_NAMES}

model_corr_count = {action_name: {keys: len(values) for keys, values in records.items()} for action_name, records in model_tracker.items() if not all([len(values)==0 for keys, values in records.items()])}
baseline_corr_count = {action_name: {keys: len(values) for keys, values in records.items()} for action_name, records in baseline_tracker.items() if not all([len(values)==0 for keys, values in records.items()])}

print("Model Correct Count", model_corr_count)
print("Baseline Correct Count", baseline_corr_count)

Model Correct Count {'calls': {'correct': 5831, 'wrong': 33}, 'raises': {'correct': 5866, 'wrong': 26}, 'folds': {'correct': 5803, 'wrong': 20}, 'checks': {'correct': 5918, 'wrong': 7}, 'allin': {'correct': 5877, 'wrong': 0}}
Baseline Correct Count {'calls': {'correct': 1733, 'wrong': 2806}, 'folds': {'correct': 5691, 'wrong': 17263}, 'allin': {'correct': 956, 'wrong': 932}}


In [7]:
model_accuracy = sum([records['correct'] for records in model_corr_count.values()]) / sum([records['correct'] + records['wrong'] for records in model_corr_count.values()])
baseline_accuracy = sum([records['correct'] for records in baseline_corr_count.values()]) / sum([records['correct'] + records['wrong'] for records in baseline_corr_count.values()])

print("Model Accuracy", model_accuracy)
print("Baseline Accuracy", baseline_accuracy)

Model Accuracy 0.997072938293455
Baseline Accuracy 0.28521833838194754


In [20]:
sample_action_name = 'calls'
sample_truth = 'correct'

model_io_records = model_tracker[sample_action_name][sample_truth]
baseline_io_records = baseline_tracker[sample_action_name][sample_truth]

# Choose a random int in the results
model_io_record = model_io_records[torch.randint(0, len(model_io_records), (1,)).item()] # Example when model is correct
baseline_io_record = baseline_io_records[torch.randint(0, len(baseline_io_records), (1,)).item()] # Example when baseline is correct

In [21]:
print('Example when model is correct')
model_io_record.display(poker_dataset)

Example when model is correct
Dataset Game Index tensor([15191])
--------------------------------
Players:
IlxxxlI
BananaMango
JloJIKa
magicriver88
KatyEverdeen
38979

Button Player: 38979

PREFLOP:
JloJIKa folds 0 
Community Cards: []
Pot Size: 0.75
Round Call Amount: 0
Player Round Bet Size: 0
Player Stack: 50.0

magicriver88 folds 0 
Community Cards: []
Pot Size: 0.75
Round Call Amount: 0
Player Round Bet Size: 0
Player Stack: 50.75

KatyEverdeen folds 0 
Community Cards: []
Pot Size: 0.75
Round Call Amount: 0
Player Round Bet Size: 0
Player Stack: 53.34

38979 folds 0 
Community Cards: []
Pot Size: 0.75
Round Call Amount: 0
Player Round Bet Size: 0
Player Stack: 50.0

IlxxxlI calls 0.25 
Community Cards: []
Pot Size: 0.75
Round Call Amount: 0
Player Round Bet Size: 0.25
Player Stack: 55.84

BananaMango checks 0 
Community Cards: []
Pot Size: 1.0
Round Call Amount: 0.5
Player Round Bet Size: 0.5
Player Stack: 61.33


FLOP:
IlxxxlI checks 0 
Community Cards: []
Pot Size: 1.0
Round Ca

In [22]:
print('Example when baseline is correct')
baseline_io_record.display(poker_dataset)

Example when baseline is correct
Dataset Game Index tensor([15790])
--------------------------------
Players:
Beep_Beep
IlxxxlI
illoke
aleks0v
roastbeans

Button Player: roastbeans

PREFLOP:
illoke raises 1.5 
Community Cards: []
Pot Size: 0.75
Round Call Amount: 0
Player Round Bet Size: 0
Player Stack: 50.0

aleks0v folds 0 
Community Cards: []
Pot Size: 2.25
Round Call Amount: 1.5
Player Round Bet Size: 0
Player Stack: 81.09

roastbeans folds 0 
Community Cards: []
Pot Size: 2.25
Round Call Amount: 1.5
Player Round Bet Size: 0
Player Stack: 27.11

Beep_Beep folds 0 
Community Cards: []
Pot Size: 2.25
Round Call Amount: 1.5
Player Round Bet Size: 0.25
Player Stack: 57.04

IlxxxlI calls 1.0 
Community Cards: []
Pot Size: 2.25
Round Call Amount: 1.5
Player Round Bet Size: 0.5
Player Stack: 51.0


FLOP:
IlxxxlI checks 0 
Community Cards: []
Pot Size: 3.25
Round Call Amount: 0
Player Round Bet Size: 0
Player Stack: 50.0

illoke bets 1.54 
Community Cards: []
Pot Size: 3.25
Round Call Amou

In [9]:
# sample_action_name = 'calls'
# sample_truth = 'correct'

# model_io_records = model_tracker[sample_action_name][sample_truth]
# baseline_io_records = baseline_tracker[sample_action_name][sample_truth]

# # Choose a random int in the results
# model_io_record = model_io_records[torch.randint(0, len(model_io_records), (1,)).item()]
# baseline_io_record = baseline_io_records[torch.randint(0, len(baseline_io_records), (1,)).item()]

# # These variable includes:
#         # self.input_tensor = input_tensor
#         # self.model_action_name = model_action_name
#         # self.model_bet_size = model_bet_size
#         # self.baseline_action_name = baseline_action_name
#         # self.baseline_bet_size = baseline_bet_size
#         # self.action_label = action_label
#         # self.bet_size_label = bet_size_label
#         # self.baseline_inputs = baseline_inputs
        
# # Baseline input is easy to view, the rest are just tensors

# model_inputs = model_io_record.baseline_inputs
# model_label_action_name = model_io_record.action_label
# model_label_bet_size = model_io_record.bet_size_label*poker_dataset.bet_std + poker_dataset.bet_mean
# model_action_name = model_io_record.model_action_name
# model_bet_size = model_io_record.model_bet_size*poker_dataset.bet_std + poker_dataset.bet_mean

# model_call_size = model_inputs['call_size']*poker_dataset.call_std + poker_dataset.call_mean
# model_pot_size = model_inputs['pot_size']*poker_dataset.pot_std + poker_dataset.pot_mean
# model_stack_size = model_inputs['stack_size']*poker_dataset.stack_std + poker_dataset.stack_mean
# model_min_raise = model_inputs['min_raise']*poker_dataset.min_raise_std + poker_dataset.min_raise_mean
# model_hole_cards = model_inputs['hole_cards']
# model_num_players = model_inputs['num_players']

# baseline_model_inputs = baseline_io_record.baseline_inputs
# baseline_label_action_name = baseline_io_record.action_label
# baseline_label_bet_size = baseline_io_record.bet_size_label*poker_dataset.bet_std + poker_dataset.bet_mean
# baseline_action_name = baseline_io_record.baseline_action_name
# baseline_bet_size = baseline_io_record.baseline_bet_size*poker_dataset.bet_std + poker_dataset.bet_mean

# baseline_call_size = baseline_model_inputs['call_size']*poker_dataset.call_std + poker_dataset.call_mean 
# baseline_pot_size = baseline_model_inputs['pot_size']*poker_dataset.pot_std + poker_dataset.pot_mean
# baseline_stack_size = baseline_model_inputs['stack_size']*poker_dataset.stack_std + poker_dataset.stack_mean
# baseline_min_raise = baseline_model_inputs['min_raise']*poker_dataset.min_raise_std + poker_dataset.min_raise_mean
# baseline_hole_cards = baseline_model_inputs['hole_cards']
# baseline_num_players = baseline_model_inputs['num_players']

# print("Model Sample Call Size", model_call_size.item())
# print("Model Sample Pot Size", model_pot_size.item())
# print("Model Sample Stack Size", model_stack_size.item())
# print("Model Sample Min Raise", model_min_raise.item())
# print("Model Sample Hole Cards", model_hole_cards)
# print("Model Sample Num Players", model_num_players.item())

# print("\nModel Label Action Name", ACTION_NAMES[model_label_action_name.argmax().item()])
# print("Model Label Bet Size", model_label_bet_size.item())
# print("\nModel Action Name", model_action_name)
# print("Model Bet Size", model_bet_size.item())

# print("\nBaseline Sample Call Size", baseline_call_size.item())
# print("Baseline Sample Pot Size", baseline_pot_size.item())
# print("Baseline Sample Stack Size", baseline_stack_size.item())
# print("Baseline Sample Min Raise", baseline_min_raise.item())
# print("Baseline Sample Hole Cards", baseline_hole_cards)
# print("Baseline Sample Num Players", baseline_num_players.item())

# print("\nBaseline Label Action Name", ACTION_NAMES[baseline_label_action_name.argmax().item()])
# print("Baseline Label Bet Size", baseline_label_bet_size.item())
# print("\nBaseline Action Name", baseline_action_name)
# print("Baseline Bet Size", baseline_bet_size.item())


